# UNLPD Crime Analysis
The goal of this project is to analyze University of Nebraska-Lincoln Police Department crime data to find trends. The next goal is to use that analysis to identify trends as they happen. Finally, the next goal is to write a program that will use those trends to write timely stories.

## Identify trends

Using pandas, I analyzed 1

In [1]:
import pandas as pd
import dateutil

In [4]:
df = pd.read_csv('original_data/all_years_new.csv')

In [5]:
df['Reported']

0             1/1/05 0:33
1             1/1/05 1:34
2            1/1/05 15:30
3            1/1/05 19:40
4            1/2/05 16:58
5            1/4/05 10:08
6            1/4/05 12:29
7            1/4/05 23:43
8             1/6/05 7:44
9            1/6/05 10:53
10           1/6/05 12:37
11            1/7/05 1:16
12            1/7/05 1:47
13           1/7/05 13:29
14           1/7/05 13:38
15           1/7/05 16:10
16            1/8/05 1:14
17           1/8/05 14:26
18           1/8/05 23:19
19            1/9/05 1:17
20            1/9/05 3:45
21          1/10/05 13:50
22          1/10/05 14:08
23          1/10/05 15:04
24          1/10/05 18:19
25          1/10/05 21:56
26          1/11/05 10:46
27          1/11/05 12:12
28           1/12/05 0:42
29          1/12/05 13:55
               ...       
24212    12/13/2017 12:05
24213    12/13/2017 12:16
24214    12/13/2017 14:11
24215    12/13/2017 14:42
24216    12/14/2017 13:27
24217    12/14/2017 17:08
24218    12/14/2017 18:45
24219     12